## Import data

In [51]:
%matplotlib inline
%reload_ext autoreload
from rtxlib.rtx_run import setup_database, db
from analysis_lib.one_sample_tests import KolmogorovSmirnov
from factorial_anova_analysis import get_raw_data

index = "rtxgaussian-600-300-baseline" 

results = get_raw_data(index, False, 5000)

data retrieved from file rtxgaussian-600-300-baseline-5000.pickle


In [52]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

knobs = {}
data = {}
for r in results:
    knob_str = str(r[1])
    if not knob_str in knobs:        
        knobs[knob_str] = []
    if len(knobs[knob_str]) < 5000:
        knobs[knob_str].append(r[1])
    if not knob_str in data:        
        data[knob_str] = []
    if len(data[knob_str]) < 5000:
        data[knob_str].append(r[0])


print "len(knobs):" + str(len(knobs))

for k in knobs:
    print k
    print len(knobs[k])

print "========="

len(knobs):20
{u're_route_every_ticks': 10, u'freshness_cut_off_value': 700, u'max_speed_and_length_factor': 1.0, u'exploration_percentage': 0.3, u'freshness_update_factor': 5, u'route_random_sigma': 0.0, u'average_edge_duration_factor': 2.5}
5000
{u're_route_every_ticks': 10, u'freshness_cut_off_value': 700, u'max_speed_and_length_factor': 2.5, u'exploration_percentage': 0.3, u'freshness_update_factor': 5, u'route_random_sigma': 0.0, u'average_edge_duration_factor': 1.0}
5000
{u're_route_every_ticks': 70, u'freshness_cut_off_value': 100, u'max_speed_and_length_factor': 1.9724309800135384, u'exploration_percentage': 0.22166267566255865, u'freshness_update_factor': 5, u'route_random_sigma': 0.2883256763888628, u'average_edge_duration_factor': 1.8526844399671079}
5000
{u're_route_every_ticks': 10, u'freshness_cut_off_value': 100, u'max_speed_and_length_factor': 1.9993660358635545, u'exploration_percentage': 0.10784601253398841, u'freshness_update_factor': 5, u'route_random_sigma': 0.1903

## Ttests with variable sample sizes

In [57]:
from analysis_lib.two_sample_tests import Ttest
from analysis_lib.two_sample_tests import TtestPower
from complaints_generator import generate_complaints
from scipy.stats import binom_test
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc("savefig", dpi=150)
import random 
from IPython.display import Markdown, display

def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))
    
color = {
    "PURPLE": '\033[95m',
    "CYAN": '\033[96m',
    "DARKCYAN": '\033[36m',
    "BLUE": '\033[94m',
    "GREEN": '\033[92m',
    "YELLOW": '\033[93m',
    "RED": '\033[91m',
    "BOLD": '\033[1m',
    "UNDERLINE": '\033[4m',
    "END": '\033[0m',
}
    
random.seed(123456)

y_key = 'overhead'
alpha = 0.05
necessary_power = 0.80

fake_run_id = "123456"

knob_values = sorted(knobs.keys())
best_knob = knob_values[0]
other_knob_values = knob_values[1:] 

sample_sizes = []
pvalues = {}
powers = {}

for i in knob_values:
    pvalues[i] = []
    powers[i] = []
        
step = 100

count = 0
for other_knob_value in other_knob_values:   
    for sample_size in range(100, 5001, step):
    #     print "Sample size: " + str(sample_size)
        sample_sizes.append(sample_size)

        analysis_data = {}
        raw_data = data[best_knob] 
#         analysis_data[0] = raw_data[:sample_size] #  also: 
        analysis_data[0] = random.sample(raw_data, sample_size)
  
        raw_data = data[other_knob_value] 
#         analysis_data[1] = raw_data[:sample_size] #  also: 
        analysis_data[1] = random.sample(raw_data, sample_size)
          
        t = Ttest(fake_run_id, y_key, alpha).start(analysis_data, {})
        pvalues[other_knob_value].append(t["pvalue"])
        one_sided_p_value = t["pvalue"]/2
        
        if t["statistic"] < 0:
            t_p = TtestPower(fake_run_id, y_key, t["effect_size"], alpha, 'smaller').start(analysis_data, {})            
        else: 
            t_p = TtestPower(fake_run_id, y_key, t["effect_size"], alpha, 'larger').start(analysis_data, {})            
        powers[other_knob_value].append(t_p["power"])
        
        if one_sided_p_value < alpha and t_p["power"] >= necessary_power:
            printmd("**SIGNIFICANT DIFFERENCE**", "green")
            different = "smaller" if t["statistic"] < 0 else "larger"
            print "sample size: " + str(sample_size) 
            print "================"
            print "Overhead in " 
            pp.pprint(eval(best_knob))
            print " is " + different + " than in "
            pp.pprint(eval(other_knob_value))
            print "================"
            print "mean difference: " + str(t["mean_diff"]) 
            print "p value: " + str(one_sided_p_value)                                 
            print "effect size: " + str(t["effect_size"]) 
            print "power: " + str(t_p["power"])
            print "\n"
            
            count += 1
            if t["statistic"] >= 0:
                best_knob = other_knob_value
            break

        if sample_size == 5000:
            printmd("**NO SIGNIFICANT DIFFERENCE**", "red")
            different = "smaller" if t["statistic"] < 0 else "larger"
            print "sample size: " + str(sample_size) 
            print "================"
            print "Overhead in " 
            pp.pprint(eval(best_knob))
            print " is " + different + " than in "
            pp.pprint(eval(other_knob_value))
            print "================"
            print "mean difference: " + str(t["mean_diff"]) 
            print "p value: " + str(one_sided_p_value)                                 
            print "effect size: " + str(t["effect_size"]) 
            print "power: " + str(t_p["power"])
            print "\n"
        
print str(count) + " out of " + str(len(other_knob_values)) + " comparisons found significant diffference."
print "******************"
print "BEST CONFIGURATION"
print "******************"
pp.pprint(knobs[best_knob][0])


<span style='color:green'>**SIGNIFICANT DIFFERENCE**</span>

sample size: 100
Overhead in 
{   u'average_edge_duration_factor': 1.0,
    u'exploration_percentage': 0.0,
    u'freshness_cut_off_value': 100,
    u'freshness_update_factor': 5,
    u'max_speed_and_length_factor': 1.0,
    u're_route_every_ticks': 10,
    u'route_random_sigma': 0.0}
 is smaller than in 
{   u'average_edge_duration_factor': 1.0,
    u'exploration_percentage': 0.3,
    u'freshness_cut_off_value': 100,
    u'freshness_update_factor': 20,
    u'max_speed_and_length_factor': 1.0,
    u're_route_every_ticks': 10,
    u'route_random_sigma': 0.0}
mean difference: -0.733624299293
p value: 1.90688817784e-09
effect size: -0.886145624588
power: 0.999999999815




<span style='color:green'>**SIGNIFICANT DIFFERENCE**</span>

sample size: 300
Overhead in 
{   u'average_edge_duration_factor': 1.0,
    u'exploration_percentage': 0.0,
    u'freshness_cut_off_value': 100,
    u'freshness_update_factor': 5,
    u'max_speed_and_length_factor': 1.0,
    u're_route_every_ticks': 10,
    u'route_random_sigma': 0.0}
 is smaller than in 
{   u'average_edge_duration_factor': 1.0,
    u'exploration_percentage': 0.3,
    u'freshness_cut_off_value': 100,
    u'freshness_update_factor': 5,
    u'max_speed_and_length_factor': 1.0,
    u're_route_every_ticks': 10,
    u'route_random_sigma': 0.3}
mean difference: -0.225512305402
p value: 0.00105628614801
effect size: -0.252601029214
power: 0.925821900653




<span style='color:green'>**SIGNIFICANT DIFFERENCE**</span>

...

sample size: 100
Overhead in 
{   u'average_edge_duration_factor': 1.209267495740946,
    u'exploration_percentage': 0.15700023908873034,
    u'freshness_cut_off_value': 632,
    u'freshness_update_factor': 6,
    u'max_speed_and_length_factor': 1.0,
    u're_route_every_ticks': 10,
    u'route_random_sigma': 0.2585412258729002}
 is smaller than in 
{   u'average_edge_duration_factor': 1.0,
    u'exploration_percentage': 0.3,
    u'freshness_cut_off_value': 100,
    u'freshness_update_factor': 20,
    u'max_speed_and_length_factor': 1.0,
    u're_route_every_ticks': 70,
    u'route_random_sigma': 0.0}
mean difference: -1.06008341239
p value: 0.000103896151275
effect size: -0.54626230531
power: 0.986257472966




<span style='color:green'>**SIGNIFICANT DIFFERENCE**</span>

sample size: 300
Overhead in 
{   u'average_edge_duration_factor': 1.209267495740946,
    u'exploration_percentage': 0.15700023908873034,
    u'freshness_cut_off_value': 632,
    u'freshness_update_factor': 6,
    u'max_speed_and_length_factor': 1.0,
    u're_route_every_ticks': 10,
    u'route_random_sigma': 0.2585412258729002}
 is smaller than in 
{   u'average_edge_duration_factor': 1.8526844399671079,
    u'exploration_percentage': 0.22166267566255865,
    u'freshness_cut_off_value': 100,
    u'freshness_update_factor': 5,
    u'max_speed_and_length_factor': 1.9724309800135384,
    u're_route_every_ticks': 70,
    u'route_random_sigma': 0.2883256763888628}
mean difference: -0.15052923573
p value: 0.00412733186786
effect size: -0.216817874124
power: 0.84317830243




<span style='color:green'>**SIGNIFICANT DIFFERENCE**</span>

sample size: 400
Overhead in 
{   u'average_edge_duration_factor': 1.209267495740946,
    u'exploration_percentage': 0.15700023908873034,
    u'freshness_cut_off_value': 632,
    u'freshness_update_factor': 6,
    u'max_speed_and_length_factor': 1.0,
    u're_route_every_ticks': 10,
    u'route_random_sigma': 0.2585412258729002}
 is smaller than in 
{   u'average_edge_duration_factor': 2.5,
    u'exploration_percentage': 0.0,
    u'freshness_cut_off_value': 100,
    u'freshness_update_factor': 5,
    u'max_speed_and_length_factor': 2.5,
    u're_route_every_ticks': 70,
    u'route_random_sigma': 0.0}
mean difference: -0.15988482992
p value: 0.00153905566137
effect size: -0.210331193728
power: 0.907771423911


17 out of 19 comparisons found significant diffference.
******************
BEST CONFIGURATION
******************
{   u'average_edge_duration_factor': 1.209267495740946,
    u'exploration_percentage': 0.15700023908873034,
    u'freshness_cut_off_value': 632,
    u'freshness_update_

In [58]:
results = get_raw_data("rtxgaussian-600-300-5", False, 5000)

b_knobs = {}
b_data = {}
for r in results:
    knob_str = str(r[1])
    if not knob_str in b_knobs:        
        b_knobs[knob_str] = []
    if len(b_knobs[knob_str]) < 5000:
        b_knobs[knob_str].append(r[1])
    if not knob_str in b_data:        
        b_data[knob_str] = []
    if len(b_data[knob_str]) < 5000:
        b_data[knob_str].append(r[0])

print "len(knobs):" + str(len(b_knobs))

for k in b_knobs:
    print k
    print len(b_knobs[k])

print "========="

data retrieved from file rtxgaussian-600-300-5-5000.pickle
len(knobs):4
{u'freshness_update_factor': 6, u'average_edge_duration_factor': 1.3437240401490982, u'exploration_percentage': 0.07526532486706616}
5000
{u'freshness_update_factor': 20, u'average_edge_duration_factor': 2.5, u'exploration_percentage': 0.3}
5000
{u'freshness_update_factor': 5, u'average_edge_duration_factor': 1.0, u'exploration_percentage': 0.3}
5000
{u'freshness_update_factor': 20, u'average_edge_duration_factor': 1.0, u'exploration_percentage': 0.0}
5000


In [59]:
b_knobs[best_knob] = knobs[best_knob]
b_data[best_knob] = data[best_knob]

print "len(knobs):" + str(len(b_knobs))

for k in b_knobs:
    print k
    print len(b_knobs[k])

print "========="

len(knobs):5
{u'freshness_update_factor': 6, u'average_edge_duration_factor': 1.3437240401490982, u'exploration_percentage': 0.07526532486706616}
5000
{u'freshness_update_factor': 20, u'average_edge_duration_factor': 2.5, u'exploration_percentage': 0.3}
5000
{u'freshness_update_factor': 5, u'average_edge_duration_factor': 1.0, u'exploration_percentage': 0.3}
5000
{u'freshness_update_factor': 20, u'average_edge_duration_factor': 1.0, u'exploration_percentage': 0.0}
5000
{u're_route_every_ticks': 10, u'freshness_cut_off_value': 632, u'max_speed_and_length_factor': 1.0, u'exploration_percentage': 0.15700023908873034, u'freshness_update_factor': 6, u'route_random_sigma': 0.2585412258729002, u'average_edge_duration_factor': 1.209267495740946}
5000


In [60]:
y_key = 'overhead'
alpha = 0.05
necessary_power = 0.80

fake_run_id = "123456"

knob_values = sorted(b_knobs.keys())
best_knob = knob_values[0]
other_knob_values = knob_values[1:] 

sample_sizes = []
pvalues = {}
powers = {}

for i in knob_values:
    pvalues[i] = []
    powers[i] = []
        
step = 100

count = 0
for other_knob_value in other_knob_values:   
    for sample_size in range(100, 5001, step):
    #     print "Sample size: " + str(sample_size)
        sample_sizes.append(sample_size)

        analysis_data = {}
        raw_data = b_data[best_knob] 
#         analysis_data[0] = raw_data[:sample_size] #  also: 
        analysis_data[0] = random.sample(raw_data, sample_size)
  
        raw_data = b_data[other_knob_value] 
#         analysis_data[1] = raw_data[:sample_size] #  also: 
        analysis_data[1] = random.sample(raw_data, sample_size)
          
        t = Ttest(fake_run_id, y_key, alpha).start(analysis_data, {})
        pvalues[other_knob_value].append(t["pvalue"])
        one_sided_p_value = t["pvalue"]/2
        
        if t["statistic"] < 0:
            t_p = TtestPower(fake_run_id, y_key, t["effect_size"], alpha, 'smaller').start(analysis_data, {})            
        else: 
            t_p = TtestPower(fake_run_id, y_key, t["effect_size"], alpha, 'larger').start(analysis_data, {})            
        powers[other_knob_value].append(t_p["power"])
        
        if one_sided_p_value < alpha and t_p["power"] >= necessary_power:
            printmd("**SIGNIFICANT DIFFERENCE**", "green")
            different = "smaller" if t["statistic"] < 0 else "larger"
            print "sample size: " + str(sample_size) 
            print "================"
            print "Overhead in " 
            pp.pprint(eval(best_knob))
            print " is " + different + " than in "
            pp.pprint(eval(other_knob_value))
            print "================"
            print "mean difference: " + str(t["mean_diff"]) 
            print "p value: " + str(one_sided_p_value)                                 
            print "effect size: " + str(t["effect_size"]) 
            print "power: " + str(t_p["power"])
            print "\n"
            
            count += 1
            if t["statistic"] >= 0:
                best_knob = other_knob_value
            break
        
        if sample_size == 5000:
            printmd("**NO SIGNIFICANT DIFFERENCE**", "red")
            different = "smaller" if t["statistic"] < 0 else "larger"
            print "sample size: " + str(sample_size) 
            print "================"
            print "Overhead in " 
            pp.pprint(eval(best_knob))
            print " is " + different + " than in "
            pp.pprint(eval(other_knob_value))
            print "================"
            print "mean difference: " + str(t["mean_diff"]) 
            print "p value: " + str(one_sided_p_value)                                 
            print "effect size: " + str(t["effect_size"]) 
            print "power: " + str(t_p["power"])
            print "\n"

print str(count) + " out of " + str(len(other_knob_values)) + " comparisons found significant difference."
print "******************"
print "BEST CONFIGURATION"
print "******************"
pp.pprint(b_knobs[best_knob][0])

<span style='color:green'>**SIGNIFICANT DIFFERENCE**</span>

sample size: 3000
Overhead in 
{   u'average_edge_duration_factor': 1.0,
    u'exploration_percentage': 0.0,
    u'freshness_update_factor': 20}
 is smaller than in 
{   u'average_edge_duration_factor': 2.5,
    u'exploration_percentage': 0.3,
    u'freshness_update_factor': 20}
mean difference: -0.0518519875995
p value: 0.00173138065698
effect size: -0.0755236462339
power: 0.899698363563




<span style='color:red'>**NO SIGNIFICANT DIFFERENCE**</span>

sample size: 5000
Overhead in 
{   u'average_edge_duration_factor': 1.0,
    u'exploration_percentage': 0.0,
    u'freshness_update_factor': 20}
 is larger than in 
{   u'average_edge_duration_factor': 1.0,
    u'exploration_percentage': 0.3,
    u'freshness_update_factor': 5}
mean difference: 0.00292140888605
p value: 0.429662074078
effect size: 0.00354513819644
power: 0.0711052705447




<span style='color:green'>**SIGNIFICANT DIFFERENCE**</span>

sample size: 500
Overhead in 
{   u'average_edge_duration_factor': 1.0,
    u'exploration_percentage': 0.0,
    u'freshness_update_factor': 20}
 is larger than in 
{   u'average_edge_duration_factor': 1.3437240401490982,
    u'exploration_percentage': 0.07526532486706616,
    u'freshness_update_factor': 6}
mean difference: 0.162562136777
p value: 0.000505569073868
effect size: 0.208923767845
power: 0.951167526059




<span style='color:red'>**NO SIGNIFICANT DIFFERENCE**</span>

sample size: 5000
Overhead in 
{   u'average_edge_duration_factor': 1.3437240401490982,
    u'exploration_percentage': 0.07526532486706616,
    u'freshness_update_factor': 6}
 is larger than in 
{   u'average_edge_duration_factor': 1.209267495740946,
    u'exploration_percentage': 0.15700023908873034,
    u'freshness_cut_off_value': 632,
    u'freshness_update_factor': 6,
    u'max_speed_and_length_factor': 1.0,
    u're_route_every_ticks': 10,
    u'route_random_sigma': 0.2585412258729002}
mean difference: 0.00546462671283
p value: 0.331932105035
effect size: 0.00869281464662
power: 0.113092965839


2 out of 4 comparisons found significant difference.
******************
BEST CONFIGURATION
******************
{   u'average_edge_duration_factor': 1.3437240401490982,
    u'exploration_percentage': 0.07526532486706616,
    u'freshness_update_factor': 6}
